<a href="https://colab.research.google.com/github/RogueRock/IDC410-ML/blob/main/ML_exercise3_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ML exercise 3
#To implement A Multi-layered Perceptron Using Numpy
#linking gdrive with colab so as to access the input file
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Q1.
#importing pandas reading the dataframe from the input file
import numpy as np
import pandas as pd
#copying the file path
file_path = "/content/gdrive/MyDrive/mnist_train.csv"
df=pd.read_csv(file_path)

In [ ]:
#checking the size and shape of the input file
size = df.size
shape = df.shape
print (size, shape)
#using iloc to slice the df and look at first_ten columns and first 200 rows
first_ten = df.iloc[0:200, :10]
print (first_ten)

47100000 (60000, 785)
     label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9
0        5    0    0    0    0    0    0    0    0    0
1        0    0    0    0    0    0    0    0    0    0
2        4    0    0    0    0    0    0    0    0    0
3        1    0    0    0    0    0    0    0    0    0
4        9    0    0    0    0    0    0    0    0    0
..     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
195      9    0    0    0    0    0    0    0    0    0
196      7    0    0    0    0    0    0    0    0    0
197      8    0    0    0    0    0    0    0    0    0
198      3    0    0    0    0    0    0    0    0    0
199      2    0    0    0    0    0    0    0    0    0

[200 rows x 10 columns]


In [ ]:
#function to take file path as input
def function_1 (file_path):
  df = pd.read_csv(file_path)
  arr1 = df.to_numpy(dtype = float) #converting to a numppy array
  x = arr1[:,1:] #removing first class cloumn
  arr2 = df.to_numpy(dtype = int)
  y = arr2[:, :1] #slicing to get the desired data
  y2 = np.transpose(y) #to get a single row
  return y2, x
y, x  = function_1 ("/content/gdrive/MyDrive/mnist_train.csv")
print (y[:,:10])
print (x.shape, y.shape)

[[5 0 4 1 9 2 1 3 1 4]]
(60000, 784) (1, 60000)


In [ ]:
#Q2
#making a class for the softmax activation function
class SoftmaxReg():
#initializing the instance
  def __init__(self):
    self.weight = np.random.randn(784, 10)/10
    self.bias = np.random.rand(1, 10)/10
   # self.softmax = softmax
    print (self.bias.shape)
#doing the forward pass and then applying the softmax function
  def forward_prop(self, x):
    self.x = x
    self.z = np.matmul(x, self.weight) + self.bias
    self.sigmas = self.softmax(self.z)
    print("Shape of sigmas:", self.sigmas.shape)
    return self.sigmas
#doing the back propagation using the cross entropy loss function derivative
#Reference ; https://jmlb.github.io/ml/2017/12/26/Calculate_Gradient_Softmax/
  def backprop(self,x, y, learn_rate):
    #self.target = y - np.log(self.sigmas)
    #loss = np.sum(-self.target)
    self.derivative1 = np.matmul(x.T, (self.sigmas - y.T))/60000
    self.derivative2 = np.mean(self.sigmas - y.T, axis=0, keepdims=True)
    self.weight = self.weight - (learn_rate*self.derivative1)/60000
    self.bias = self.bias + (learn_rate*self.derivative2)/60000
    return self.weight, self.bias
  def softmax(self,z):
    max_z = np.max(self.z, axis=1, keepdims=True)
    #<ipython-input-28-9f6efb757d7b>:23: RuntimeWarning: overflow encountered in exp
    #therefore using the max to subtract from logits of z, log-sum-exp approach
    #Reference : https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
    #e = np.exp(self.z)
    e = np.exp(self.z - max_z)
    e_sum = np.sum(e, axis = 1, keepdims = True)
    sigma = e/e_sum
    return sigma

model1 = SoftmaxReg()
learn_rate = 0.001
# forward prop
sigmas = model1.forward_prop(x)

# back prop
updated_weights, updated_bias = model1.backprop(x, y, learn_rate)

# printing for debugging
print("Updated weights shape:", updated_weights.shape)
print("updated bias shape:", updated_bias.shape )

(1, 10)
Shape of sigmas: (60000, 10)
Updated weights shape: (784, 10)
updated bias shape: (1, 10)


In [ ]:
#Q3.
#making a class for adding just one hidden layer of 16 neurons
class HiddenLayer1 ():

  def __init__ (self, x, softmax):
    self.x = x
    self.softmax = softmax
    #self.sigmoid = sigmoid
  #def generator (self):
    self.weight1 = np.random.randn(784, 16)
    self.bias1 = np.random.rand(1, 16)
    self.weight2 = np.random.randn(16, 10)
    self.bias2 = np.random.rand(1, 10)
    self.activation_2 = self.sigmoid
    self.activation_3 = self.softmax

  '''def layer_size (self, x, hidden_shape, output_shape):
    self.inputshape = self.x.shape[0]
    self.hlshape = hidden_shape
    self.out_shape = output_shape'''

  def forward_prop(self, x):
    self.x = x
    self.z1 = np.dot(self.x, self.weight1) + self.bias1
    self.y_hat1 = self.activation_2(self.z1)
    self.z2 = np.dot(self.y_hat1, self.weight2) + self.bias2
    self.y_hat2 = self.activation_3(self.z2) #self.softmax(self.z2)
    print("Shape of y_hat1:", self.y_hat1.shape)
    return self.y_hat2

  def back_prop(self, y, learn_rate):
    dz1 = (self.y_hat2 - y.T)
    #gradient for weights 1
    self.deri_weight2 = np.sum(np.matmul(self.z2.T, dz1))/60000
    #dividing for normalization
    self.deri_bias2 = np.mean(self.y_hat2 - y.T, axis=0, keepdims=True)
    #gradient for weights 1
    self.deri_weight1 = np.sum(np.matmul(self.x.T, dz1))/60000
    #self.deri_bias1 = np.mean(self.y_hat2 - y.T, axis=0, keepdims=True)
    #updating weights
    self.weight1 = self.weight1 - (learn_rate*self.deri_weight1)/60000
    #self.bias1 = self.bias1 - (learn_rate*self.deri_bias1)/60000
    self.weight2 = self.weight2 - (learn_rate*self.deri_weight2.T)/60000
    #self.bias2 = self.bias1 - (learn_rate*self.deri_bias2)/60000
    return self.weight1, self.weight2, #self.bias1, self.bias2

  def sigmoid (self, z):
    z = np.dot(self.x, self.weight1) + self.bias1
    sigma = 1 / (np.exp(z) + 1)
    return sigma
  #def sigmoid_deri (self, z):
   # return self.x * (1 - self.x)

model2 = HiddenLayer1(x, model1.softmax)
learn_rate = 0.001
#forward prop
y_hat2 = model2.forward_prop(x)

# back prop
updated_weights1, weight2 = model2.back_prop(y, learn_rate)

# printing for debugging
print("Updated weights shape:", updated_weights1.shape, weight2.shape)
#print("updated bias shape:", bias1.shape, bias2.shape )

<ipython-input-110-340b3e954109>:50: RuntimeWarning: overflow encountered in exp
  sigma = 1 / (np.exp(z) + 1)


Shape of y_hat1: (60000, 16)
Updated weights shape: (784, 16) (16, 10)


In [ ]:
#Q4.
# making a class for adding two hidden layers
class NeuralNetwork ():

  def __init__ (self,x, softmax, sigmoid):
    self.x = x
    self.softmax = softmax
    self.sigmoid = sigmoid
    #self.relu = relu
    #self.tanh = tanh
    #self.sigmoid_deri = sigmoid_deri
    self.weight1 = np.random.randn(784, 16)
    self.bias1 = np.random.rand(1, 16)
    self.weight2 = np.random.randn(16, 16)
    self.bias2 = np.random.rand(1, 16)
    self.weight3 = np.random.randn(16, 10)
    self.bias3 = np.random.rand(1, 10)
    self.activation_2 = self.sigmoid
    self.activation_3 = self.softmax

  def forward_prop (self,x):
    self.x = x
    self.z1 = np.dot(self.x, self.weight1) + self.bias1
    self.y_hat1 = self.activation_2(self.z1)
    self.z2 = np.dot(self.y_hat1, self.weight2) + self.bias2
    self.y_hat2 = self.activation_2(self.z2) #self.softmax(self.z2)
    self.z3 = np.dot(self.y_hat2, self.weight3) + self.bias3
    self.y_hat3 = self.activation_3(self.z3)
    print("Shape of y_hat2:", self.y_hat2.shape)
    return self.y_hat3

  def back_prop (self, y, learn_rate):
    #gradients for weights
    self.deri_weight3 = np.sum(np.matmul(self.z3.T, (self.y_hat3 - y.T)))/60000
    #self.deri_bias2 = np.mean(self.y_hat2 - y.T, axis=0, keepdims=True)
    dz1 = (self.y_hat3 - y.T)
    self.deri_weight2 = np.sum(np.matmul(self.z2.T, dz1))/60000
    self.deri_weight1 = np.sum(np.matmul(self.x.T, dz1))/60000
    #self.dervative1 = np.mean(self.sigmas - y.T, axis=0, keepdims=True)
    #updating the weights and the bias
    self.weight1 = self.weight1 - (learn_rate*self.deri_weight1)/60000
    #self.bias1 = self.bias1 - (learn_rate*self.derivative2)/60000
    self.weight2 = self.weight2 - (learn_rate*self.deri_weight2.T)/60000
    #self.bias2 = self.bias2 - (learn_rate*self.derivative2)/60000
    self.weight3 = self.weight3 - (learn_rate*self.deri_weight3.T)/60000
    #self.bias3 = self.bias3  (learn_rate*self.derivative2)/60000
    return self.weight1, self.weight2, self.weight3


model3 = NeuralNetwork(x, model1.softmax, model2.sigmoid)
learn_rate = 0.001
#forward prop
y_hat3 = model3.forward_prop(x)

#back prop
updated_weights1, weight2, weight3 = model3.back_prop(y, learn_rate)

# printing for debugging
print("Updated weights shape:", updated_weights1.shape, weight2.shape, weight3.shape)

<ipython-input-110-340b3e954109>:50: RuntimeWarning: overflow encountered in exp
  sigma = 1 / (np.exp(z) + 1)


Shape of y_hat2: (60000, 16)
Updated weights shape: (784, 16) (16, 16) (16, 10)


In [ ]:
#we can write this class with the activation functions defined as methods
#then we can pass the instances of this class to the other classes
'''class ActivationFunction ():

 def __init__ (self,x,z):
  self.x = x
  self.z = z
 def relu(self,z):
  delta = np.maximum(0,self.x)
  return delta

 def softmax(self,z):
    max_z = np.max(self.z, axis=1, keepdims=True)
    #<ipython-input-28-9f6efb757d7b>:23: RuntimeWarning: overflow encountered in exp
    #therefore using the max to subtract from logits of z, log-sum-exp approach
    #Reference : https://gregorygundersen.com/blog/2020/02/09/log-sum-exp/
    e = np.exp(self.z)
    e = np.exp(self.z - max_z)
    e_sum = np.sum(e, axis = 1, keepdims = True)
    sigma = e/e_sum
    return sigma

 def sigmoid (self, z):
    self.z = np.dot(self.x, self.weight1) + self.bias1
    sigma = 1 / (np.exp(-z) + 1)
    return sigma

 def tanh (self):
    f = (2/(np.exp(-2 * self.x) + 1)) - 1
    return f

activation_func = ActivationFunction(x,z)'''

In [ ]:
#Q5.
class NNetwork ():
#initializing the weights and the bias
#then making the instances of the activation functions
 def __init__ (self, x, softmax, sigmoid, back_prop):
  self.x = x
  self.softmax = softmax
  self.sigmoid = sigmoid
  self.back_prop = back_prop
  self.weight1 = np.random.randn(784, 16)
  self.bias1 = np.random.rand(1, 16)
  self.weight2 = np.random.randn(16, 16)
  self.bias2 = np.random.rand(1, 16)
  self.weight3 = np.random.randn(16, 10)
  self.bias3 = np.random.rand(1, 10)
  self.activation_2 = self.sigmoid
  self.activation_3 = self.softmax
  self.activation_4 = self.relu
  self.activation_5 = self.tanh
  #self.sigmoid_deri = sigmoid_deri
#doing forward propagation
 def forward_prop(self):
    self.x = x
    self.z1 = np.dot(self.x, self.weight1) + self.bias1
    self.y_hat1 = self.activation_2(self.z1) #sigmoid
    self.z2 = np.dot(self.y_hat1, self.weight2) + self.bias2
    self.y_hat2 = self.activation_4(self.z2) #ReLU
    self.z3 = np.dot(self.y_hat2, self.weight3) + self.bias3
    self.y_hat3 = self.activation_3(self.z3) #softmax for the output
    print("Shape of y_hat2:", self.y_hat2.shape)
    return self.y_hat3
#using ReLU activation function
 def relu(self,z):
  delta = np.maximum(0,self.x)
  return delta
 def tanh (self):
    f = (2/(np.exp(-2 * self.x) + 1)) - 1
    return f

model4 = NNetwork (x, model1.softmax, model2.sigmoid, model3.back_prop)

updated_weights1, weight2, weight3 = model4.back_prop(y, learn_rate)

# printing for debugging
print("Updated weights shape:", updated_weights1.shape, weight2.shape, weight3.shape)


Updated weights shape: (784, 16) (16, 16) (16, 10)


In [ ]:
#Q6.
class MomentumGD (NNetwork):

  def __init__ (self, x, epoch):
     super().__init__(x, model1.softmax, model2.sigmoid, model3.back_prop)
     self.x = x
     self.epoch = epoch
     #initialzing the velocities
     self.v1 = 0
     self.v2 = 0
     self.v3 = 0
#we can define this method for momentum
  '''def momentum (self, beta, learn_rate):
    self.v1 = self.v1 * beta - (1-beta)*self.gradw1
    self.v2 = self.v2 * beta - (1-beta)*self.gradw2
    self.v3 = self.v3 * beta - (1-beta)*self.gradw3
    self.weight1 = self.weight1 - (learn_rate * self.v1)
    self.weight2 = self.weight2 - (learn_rate * self.v2)
    self.weight3 = self.weight3 - (learn_rate * self.v3)
    return'''

  def back_prop1(self, y):
    self.gradw3 = np.sum(np.matmul(self.z3.T, (self.y_hat3 - y.T)))/60000
    #self.deri_bias2 = np.mean(self.y_hat2 - y.T, axis=0, keepdims=True)
    dz1 = (self.y_hat3 - y.T)
    self.gradw2 = np.sum(np.matmul(self.z2.T, dz1))/60000
    self.gradw1 = np.sum(np.matmul(self.x.T, dz1))/60000
    #self.dervative1 = np.mean(self.sigmas - y.T, axis=0, keepdims=True)
    #self.weight1 = self.weight1 - (learn_rate*self.deri_weight1)/60000
    #self.bias1 = self.bias1 - (learn_rate*self.derivative2)/60000
    #self.weight2 = self.weight2 - (learn_rate*self.deri_weight2.T)/60000
    #self.bias2 = self.bias2 - (learn_rate*self.derivative2)/60000
    #self.weight3 = self.weight3 - (learn_rate*self.deri_weight3.T)/60000
    #self.bias3 = self.bias3  (learn_rate*self.derivative2)/60000
    return self.gradw1, self.gradw2, self.gradw3


  def train (self,beta):
    for i in range(epoch):
#forward prop
        self.x = x
        self.z1 = np.dot(self.x, self.weight1) + self.bias1
        self.y_hat1 = self.activation_2(self.z1)
        self.z2 = np.dot(self.y_hat1, self.weight2) + self.bias2
        self.y_hat2 = self.activation_2(self.z2) #self.softmax(self.z2)
        self.z3 = np.dot(self.y_hat2, self.weight3) + self.bias3
        self.y_hat3 = self.activation_3(self.z3)
# back prop
        self.gradw1, self.gradw2, self.gradw3 = self.back_prop1(y)
# momentum update
        #self.momentum(beta, learn_rate)
        self.v1 = self.v1 * beta - (1-beta)*self.gradw1
        self.v2 = self.v2 * beta - (1-beta)*self.gradw2
        self.v3 = self.v3 * beta - (1-beta)*self.gradw3
        '''self.vbias1 = self.v1 * beta - (1-beta) *self.gradb1'''
        #updating the weights
        self.weight1 = self.weight1 - (learn_rate * self.v1)
        self.weight2 = self.weight2 - (learn_rate * self.v2)
        self.weight3 = self.weight3 - (learn_rate * self.v3)
        '''self.bias1 = self.bias1 - (learn_rate * self.vbias1)'''
        pass

    return self.weight1, self.weight2, self.weight3
#beta is the momentum hyperparameter
beta = 0.9
epoch = 10
model5 = MomentumGD(x, epoch)
updated_weights1, weight2, weight3 = model5.train(beta)

#printing for debugging
print("Updated weights shape:", updated_weights.shape, weight2.shape, weight3.shape)


<ipython-input-110-340b3e954109>:50: RuntimeWarning: overflow encountered in exp
  sigma = 1 / (np.exp(z) + 1)


Updated weights shape: (784, 10) (16, 16) (16, 10)
